In [1]:
import datetime
import requests
import json
import time
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import pickle
import tensorflow

from bs4 import BeautifulSoup

In [10]:
class CryptocurrencyPay :
    code_list = ['ADA', 'XTZ', 'XRP', 'XLM', 'DOGE', 'MATIC', 'AXS', 'ETC',
                 'HBAR', 'LINK', 'VET', 'SOL', 'BTC', 'ETH', 'TRX', 'CRO',
                 'THETA', 'DOT', 'MANA', 'LTC']
    name_list = ['Cardano', 'Tezos', 'XRP', 'stellar', 'Dogecoin', 'Polygon', 'Axie-Infinity', 'Ethereum-Classic',
                 'Hedera', 'Chainlink', 'Vechain', 'Solana', 'Bitcoin', 'Ethereum', 'Tron', 'Crypto-com-coin',
                 'theta-network', 'polkadot-new', 'decentraland', 'litecoin']
    ex_rate = fdr.DataReader('USD/KRW', datetime.datetime.now()).Close.values[0]
    gas_fee = {
    'ADA' : 0.3,
    'XTZ' : 0.05, 
    'XRP' : 0.01, 
    'XLM' : 0.0, 
    'DOGE' : 0.34, 
    'MATIC' : 0.002, 
    'AXS' : 4.65,
    'ETC' : 0.0033,
    'HBAR' : 0.0001, 
    'LINK' : 0.004, 
    'VET' : '0.2', 
    'SOL' : 0.00025, 
    'BTC' : 3,
    'ETH' : 6,
    'TRX' : '0.1',
    'CRO' : '0.1',
    'THETA' : '0.2', 
    'DOT' : 0, 
    'MANA' : '0.2', 
    'LTC' : 0.03,
    }
    today = datetime.datetime.now()
    dummy = None
    usd = None
    
    def __init__(self, my_currencies = dict(zip(code_list,np.random.randint(0, 100, len(code_list))))):
        self.my_currencies = my_currencies
        
        
    def update_my_currencies(self, currencies):
        self.my_currencies = currencies
    
    
    def current_upbit_prices(self, codes = code_list):
        coin_dict = {}
        for code in codes :
            url = f"https://api.upbit.com/v1/candles/minutes/1?market=KRW-{code}&count=1"
            headers = {"Accept": "application/json"}
            response = requests.request("GET", url, headers=headers)
            coin_dict[code] = response.json()[0]['trade_price']

            time.sleep(0.2)
        return json.dumps(coin_dict)
    
    
    def prev_20_upbit_prices(self, code):
        temp_df = None
        url = "https://api.upbit.com/v1/candles/days?"
        headers = {"Accept": "application/json"}
        params = {
            "market" : 'KRW-' + code,
            "count" : 21,
        }

        try : 
            response = requests.request("GET", url, headers=headers, params=params)
            if response.status_code == 200 :
                temp_df = pd.read_json(response.text)

                temp_df = temp_df[['candle_date_time_utc', 'trade_price']].sort_values('candle_date_time_utc')
                temp_df.columns = ['Date', 'Upbit_price']
                temp_df['Date'] = pd.to_datetime(temp_df['Date'])
                temp_df.set_index('Date', inplace=True)
            else :
                print(response)
                
        except Exception as e:
            print(e)

        return temp_df
    
    
    def merge_prev_20_exchange_rate(self, df):
        if self.usd is None :
            self.usd = fdr.DataReader('USD/KRW', self.today - datetime.timedelta(days = 25))['Close']
        return pd.merge(df, self.usd,how='left', left_index=True, right_index=True).ffill().bfill()
    
    
    def current_investing_prices(self, codes = code_list):
        url = 'https://www.investing.com/crypto/currencies'
        header = {
          "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
          "X-Requested-With": "XMLHttpRequest"
        }
        res = requests.get(url, headers = header)

        soup = BeautifulSoup(res.text, 'lxml')

        code_dict = {}
        for tag in soup.select('tbody > tr'):
            code = tag.select('td.js-currency-symbol')[0].text
            if code in codes :
                price = tag.select('td.js-currency-price')[0].text
                code_dict[code] = float(price.replace(',', ''))
                
        return code_dict
    
    
    def prev_20_investing_prices(self, code):
        name = self.name_list[self.code_list.index(code)].lower()
        
        temp_dict = {
            'Date' : [],
            'Price_investing' : [],
        }

        url = f'https://www.investing.com/crypto/{name}/historical-data'
        headers = {
            "User-Agent" : '*',
            "X-Requested-With": "XMLHttpRequest"
        }

        res = requests.get(url, headers = headers)
        soup = BeautifulSoup(res.text, 'lxml')

        for tag in soup.select('table.historicalTbl > tbody > tr'):
            temp = tag.text.split('\n')
            temp_dict['Date'].append(temp[1])
            temp_dict['Price_investing'].append(float(temp[2].replace(',', '')))
            
        df = pd.DataFrame(temp_dict)
        df['Date'] = pd.to_datetime(df['Date'])
        
        return df.set_index('Date')
    
    def apply_categorical(self, sr) :
        if -0.01 <= sr < 0.01 :
            return 0
        elif 0.01 <= sr < 0.05 :
            return 1
        elif 0.05 <= sr < 0.10 :
            return 2
        elif 0.10 <= sr < 0.20 :
            return 3
        elif 0.20 <= sr :
            return 4
        elif -0.05 <= sr < -0.01 :
            return 5
        elif -0.10 <= sr < -0.05 :
            return 6
        elif -0.20 <= sr < -0.10 :
            return 7
        elif sr < -0.20 :
            return 8
    
    
    def predict_tommorow_price(self):
        self.updown_predict = pickle.load(open('rfc_ud', 'rb'))
        self.change_predict = pickle.load(open('rfc', 'rb'))
        
        res_dict = {}
        for code in self.my_currencies.keys() :
            df = pd.merge(self.merge_prev_20_exchange_rate(self.prev_20_upbit_prices(code)),
                          self.prev_20_investing_prices(code),
                          how='left', left_index=True, right_index=True)
            df['KimP'] = df.Upbit_price / (df.Price_investing * df.Close)
            df['Upbit_price_c'] = df.Upbit_price.pct_change()
            df.dropna(inplace=True)
            X = tensorflow.keras.utils.to_categorical(df.Upbit_price_c.apply(self.apply_categorical), num_classes=9)
            X = X.reshape(-1)
            X = pd.concat([pd.DataFrame(X), df.KimP]).values
            X = pd.DataFrame(X)
            X = X.T

            ud = self.updown_predict.predict(X)[0]
            ch = self.change_predict.predict(X)[0]
            res_dict[code] = [ud, ch]

        return res_dict
            
            
    def dummy_prices(self, to_dict=False):
        upbit = self.current_upbit_prices()
        upbit = eval(upbit)
        investing = self.current_investing_prices()
        
        df1 = pd.DataFrame(data=upbit.values(), index = upbit.keys(), columns = ['Upbit'])
        df2 = pd.DataFrame(data=investing.values(), index = investing.keys(), columns = ['Investing'])
        dummy = pd.merge(df1, df2, left_index=True, right_index=True)
        
        dummy['Upbit_dollar'] = dummy.Upbit / self.ex_rate
        rand = np.random.randn(len(dummy))
        dummy['Upbit_dum'] = dummy.Upbit_dollar * (1 + (rand / 100))
        rand = np.random.randn(len(dummy))
        dummy['Investing_dum1'] = dummy.Investing * (1 + (rand / 100))
        rand = np.random.randn(len(dummy))
        dummy['Investing_dum2'] = dummy.Investing * (1 + (rand / 100))
        
        self.dummy = dummy
        
        if to_dict :
            return dummy.to_dict('index')
        else :
            return dummy
    
    
    def apply_gasfee_and_exrate(self, price, gasfee, amount=1):
        if type(gasfee) == str :
            return round(amount * price * (1 + float(gasfee)) * self.ex_rate, 1)

        return round((amount * price + gasfee) * self.ex_rate, 1)
    
    
    def get_optimal_market(self):
        if self.dummy is None :
            self.dummy = self.dummy_prices()
            
        res = {}
        profit = 0
        
        for code in self.dummy.index :
            index = code
            column = self.dummy.columns[self.dummy.loc[code].argmin()]
            left = self.dummy.loc[index]['Upbit'] * self.my_currencies[code]
            right = self.apply_gasfee_and_exrate(self.dummy.loc[index][column],
                                                 amount=self.my_currencies[code], gasfee=self.gas_fee[code])
            
            if left >= right :
                res[code] = 'upbit'
                
            else :
                res[code] = column, ((right-left) / self.my_currencies[code])
                profit += right-left
            
        return res
    
    def get_optimal_values(self):

        optimal = self.get_optimal_market()
        
        preds = self.predict_tommorow_price()
        priority_coin = {}
        spread = {}

        for code in self.my_currencies.keys():
            value = float(self.dummy.Upbit[code])
            if preds[code][0] == 0:
                if preds[code][1] == 0 :
                    if type(optimal[code]) == tuple :
                        value += optimal[code][1]
                        spread[code] = optimal[code][1]
                    priority_coin[code] = value * 1.15 / float(self.dummy.Upbit[code])

                elif preds[code][1] == 1 :
                    if type(optimal[code]) == tuple :
                        value += optimal[code][1]
                        spread[code] = optimal[code][1]
                    priority_coin[code] =  value * 1.03 / float(self.dummy.Upbit[code])

            else :
                if preds[code][1] == 1 :
                    if type(optimal[code]) == tuple :
                        value += optimal[code][1]
                        spread[code] = optimal[code][1]
                    priority_coin[code] =  value * 0.97 / float(self.dummy.Upbit[code])

                else :
                    if type(optimal[code]) == tuple :
                        value += optimal[code][1]
                        spread[code] = optimal[code][1] 
                    priority_coin[code] =  value * 0.85 / float(self.dummy.Upbit[code])

        sorte = dict(sorted(priority_coin.items(), key=lambda item: item[1], reverse=True))

        df_dict = {
            'Price' : pay.dummy.Upbit,
            'Amount' : pay.my_currencies,
            'Priority' : priority_coin,
            'Spread' : spread
        }
        df = pd.DataFrame(df_dict)
        df['Value'] = df.Price * df.Amount
        return df.sort_values(['Priority', 'Value'], ascending=False).fillna(0)
    
    
    def use_currency(self, price):
        df = self.get_optimal_values()
        total = 0
        spread = 0
        br = False
        use = {}
        for currency in df.iloc :
            use[currency.name] = 0
            for amount in range(1, int(currency['Amount']+1)):
                total += currency['Price']
                spread += currency['Spread'] / (currency['Amount'] + 1)
                use[currency.name] += 1
                if total >= price :
                    return (use, spread, total - price)
            if br :
                break
        return '자산이 부족합니다.', total

In [15]:
pay = CryptocurrencyPay()

In [16]:
pay.current_upbit_prices()

'{"ADA": 1645.0, "XTZ": 5520.0, "XRP": 1030.0, "XLM": 354.0, "DOGE": 209.0, "MATIC": 3015.0, "AXS": 116400.0, "ETC": 42580.0, "HBAR": 376.0, "LINK": 26230.0, "VET": 104.0, "SOL": 210400.0, "BTC": 57134000.0, "ETH": 4625000.0, "TRX": 93.5, "CRO": 703.0, "THETA": 5815.0, "DOT": 34960.0, "MANA": 4000.0, "LTC": 181950.0}'

In [17]:
pay.predict_tommorow_price()

{'ADA': [1, 1],
 'XTZ': [1, 1],
 'XRP': [0, 1],
 'XLM': [1, 1],
 'DOGE': [1, 1],
 'MATIC': [1, 1],
 'AXS': [0, 1],
 'ETC': [1, 1],
 'HBAR': [0, 1],
 'LINK': [1, 1],
 'VET': [0, 1],
 'SOL': [1, 1],
 'BTC': [0, 1],
 'ETH': [0, 1],
 'TRX': [0, 1],
 'CRO': [0, 1],
 'THETA': [0, 1],
 'DOT': [1, 1],
 'MANA': [1, 1],
 'LTC': [0, 1]}

In [18]:
pay.dummy_prices()

,Upbit,Investing,Upbit_dollar,Upbit_dum,Investing_dum1,Investing_dum2
ADA,1640.0,1.349300,1.374905,1.353396,1.345560,1.338893
XTZ,5520.0,4.552640,4.627728,4.669943,4.544781,4.551338
XRP,1025.0,0.843860,0.859315,0.852591,0.853677,0.856786
XLM,354.0,0.291410,0.296778,0.297076,0.292444,0.286852
DOGE,209.0,0.172035,0.175217,0.174098,0.174128,0.169137
MATIC,3020.0,2.481000,2.531837,2.511397,2.440524,2.490346
AXS,116350.0,95.848200,97.542777,98.483830,95.890133,96.132695
ETC,42590.0,34.986800,35.705603,35.006198,35.344304,35.300039
HBAR,378.0,0.310610,0.316899,0.321728,0.310982,0.308707
LINK,26250.0,21.600000,22.006858,21.773500,21.215244,21.222530


In [20]:
%%time
pay.use_currency(1200000)

CPU times: user 4.1 s, sys: 456 ms, total: 4.56 s
Wall time: 23.6 s


({'THETA': 29,
  'VET': 14,
  'MANA': 13,
  'TRX': 51,
  'CRO': 91,
  'DOGE': 18,
  'BTC': 1},
 1574.8607800856694,
 56277657.7)

In [14]:
from flask import Flask

from flask_cors import CORS

pay = CryptocurrencyPay()
app = Flask(__name__)
CORS(app)


@app.get('/upbit_price')
def get_upbit_prices():
    res = pay.current_upbit_prices()
    return res

# @app.get('')

if __name__ == '__main__':
    app.run(host='127.0.0.1', port='8080')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2022 15:26:16] "GET /upbit_price HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 15:26:22] "GET /upbit_price HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 15:27:14] "GET /upbit_price HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 15:27:20] "GET /upbit_price HTTP/1.1" 200 -
